# Introduction

This tutorial will walk you through a preliminary similarity searching analysis making use of scripts in the AMOEBAE toolkit. While AMOEBAE was not originally written to be used via the command line, Jupyter notebooks provide an easy means of guiding new users through an example analysis with limited need for manual input. The end result of running this code successfully is a spreadsheet summarizing results of similarity searches, as well as a plot to visualize the results.

As a simple example, we will consider the the distribution of orthologues of subunits of the Adaptor Protein (AP) 2 vesicle adaptor complex, and several other membrane-trafficking proteins, in five model eukaryotes: the plant *Arabidopsis thaliana*, the yeast *Saccharomyces cerevisiae*, the fungus *Allomyces macrogynus*, the amoeba *Dictyostelium discoideum*, and the pathogenic protist *Trypanosoma brucei*. AP-2 subunits are homologous to subunits of other AP complexes (Robinson, 2004; Hirst et al., 2011), and published work has traced their evolution among plants (Larson et al., 2019), Fungi (Barlow et al., 2014), and trypanosomatid parasites (Manna et al., 2013). Thus, the protein subunits of the AP-2 complex provide a useful test of similarity searching methods to distinguish between orthologues and paralogues, which can be compared to the results of previous studies. In addition, the membrane trafficking proteins Sec12 (a component of the COPII vesicle coat complex), SNAP33 (a Qbc-SNARE), and Rab2 (a small GTPase) are included to further explore the potential sources of error involved in identification of orthologous proteins.

## Objectives of this tutorial


-  Perform similarity searches using the BLASTP, TBLASN, HMMer algorithms simultaneously using AMOBEAE scripts.

-  Apply a reciprocal-best-hit search strategy using AMOEBAE code.

- Practice interpreting similarity search results obtained using AMOEBAE.


## Requirements

- Before running this code, you will need to have set up AMOEBAE according to the instructions in the main documentation file here (which you likely have already done): [AMOEBAE_documentation.pdf](
https://github.com/laelbarlow/amoebae/blob/master/documentation/AMOEBAE_documentation.pdf).

- MacOS or Linux operating system (or possibly a work-around on windows, although this has not been tested).

- Approximately 3GB of storage space.

- An internet connection.

- At least an hour of your time (the code in this notebook will take approximately 60 minutes to run).

- Running the code in this notebook is more computationally intensive than webbrowsing for example, so if you are running this on a laptop computer, then make sure it is connected to an electrical outlet.

## Testing
If you wish to simply run all the code in this notebook for testing purposes: Select "Cell" > "Run All" from the Jupyter menu above.

# Preliminary steps

## Find the amoebae script

The directory containing the amoebae executable script must be present in your $PATH.

In [1]:
%%bash
printf "\nThis is the directory that this notebook is run in:\n"
pwd
echo
printf "\nThis is the path to the amoebae executable script that will be used:\n"
command -v amoebae
#echo
#printf "\nThese are all the paths in the \$PATH variable:\n"
#tr ':' '\n' <<< "$PATH"


This is the directory that this notebook is run in:
/home/vagrant/amoebae/notebooks


This is the path to the amoebae executable script that will be used:
/opt/amoebae/amoebae


## Check that dependencies are installed

You should have already pulled the amoebae git repository to your computer as described in the main documentation file.

In [2]:
%%bash
# This command simply prints the versions of some dependencies which are now available for use by amoebae.
amoebae check_depend



BLASTP version:
blastp: 2.9.0+


HMMer version:
# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.3 (Nov 2019); http://hmmer.org/
# Copyright (C) 2019 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.


HMMer esl-fetch utilities:
# esl-sfetch :: retrieve sequence(s) from a file
# Easel 0.46 (Nov 2019)
# Copyright (C) 2019 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.


MUSCLE version:
MUSCLE v3.8.31 by Robert C. Edgar


IQ-TREE version:
IQ-TREE multicore version 1.6.12 for Linux 64-bit built Aug 15 2019




In [3]:
%%bash
# This command tests all the import statements in amoebae modules.
amoebae check_imports


Non-redundant list of import statements:

1. import sys  # add_seq_man.py
2. import os  # add_seq_man.py
3. import shutil  # add_seq_man.py
4. import time  # add_seq_man.py
5. from afa_to_nex import afa_to_nex, nex_to_afa  # add_seq_man.py
6. from afa_to_fa import afa_to_fa  # add_seq_man.py
7. from afa_to_nex import align_one_fa  # add_seq_man.py
8. from subprocess import call  # add_seq_man.py
9. from parse_mod_num import update_mod_num_numeric  # add_seq_man.py
10. import subprocess  # boots_on_best_ml.py
11. import glob  # boots_on_best_ml.py
12. import settings  # boots_on_best_ml.py
13. from name_replace import write_newick_tree_with_uncoded_names  # boots_on_best_ml.py
14. from trim_nex import trim_nex  # trim_nex_script.py
15. import math  # calculate_possible_seq_sets_to_tree.py
16. import argparse  # run_rev_srch.py
17. import run_searches  # run_rev_srch.py
18. import re  # trim_commas_csv.py
19. from get_fas_from_db_dir import get_seq_obj_from_db_fasta  # nex_to_full-lengt

## Import some basic python modules

In [4]:
import os
import sys
import platform
import subprocess
from Bio import SeqIO
import glob
from Bio.Blast import NCBIXML
import pandas as pd
from IPython.display import display, HTML, Image
import requests

## Update PATH so that additional modules can be imported.

In [5]:
!echo $PATH

/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/conda/bin:/opt/ncbi-blast-2.9.0+/bin:/opt/iqtree-1.6.12-Linux/bin:/opt/:/opt/amoebae


In [6]:
# Add parent directory (the main amoebae repository directory) to the $PATH.
sys.path.append('..')
!echo $PATH

/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/opt/conda/bin:/opt/ncbi-blast-2.9.0+/bin:/opt/iqtree-1.6.12-Linux/bin:/opt/:/opt/amoebae


In [7]:
import settings

## Record the specific version of AMOEBAE code used

In [8]:
# Record git repository version information.
wd = !pwd
script_dir = wd[0] 
git_hash = str(subprocess.check_output(["git", "rev-parse", "HEAD"], cwd=script_dir).strip())
git_branch = str(subprocess.check_output(["git", "rev-parse", "--abbrev-ref", "HEAD"], cwd=script_dir).strip())  
print('\nGit repository (code) version: ' + git_hash + ' (branch name: ' + git_branch + ')\n')


Git repository (code) version: b'739a3aaec4a730425e26eab0ee0e31e35cb3531c' (branch name: b'master')



## Make a subdirectory to store output.

In [9]:
%%bash
mkdir amoebae_tutorial_2_output

In [10]:
%cd amoebae_tutorial_2_output

/home/vagrant/amoebae/notebooks/amoebae_tutorial_2_output


# Set up sequence databases for searching

## Download peptide and nucleotide sequences for specific genomes.

Let's download the predicted peptide sequences, genomic assembly (nucleotide
sequences of assembled chromosomes), and annotation files (in GFF3 format) for the following eukaryotes from NCBI:

- *Arabidopsis thaliana*
- *Trypanosoma brucei*
- *Dictyostelium discoideum*
- *Allomyces macrogynus*
- *Saccharomyces cerevisiae*


This will take approximately 2 minutes.

In [11]:
%%time

# Initiate a list of file paths for downloaded sequence and annotation files.
datafile_path_list = []

# Define a dictionary of source URLs and new filenames for sequence and annotation files.
# Note that the filenames (besides extension) are the species name with underscores instead of spaces.
datafile_dict = {"Arabidopsis_thaliana.faa": "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/735/GCF_000001735.4_TAIR10.1/GCF_000001735.4_TAIR10.1_protein.faa.gz",
                 "Arabidopsis_thaliana.fna": "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/735/GCF_000001735.4_TAIR10.1/GCF_000001735.4_TAIR10.1_genomic.fna.gz",
                 "Arabidopsis_thaliana.gff3": "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/735/GCF_000001735.4_TAIR10.1/GCF_000001735.4_TAIR10.1_genomic.gff.gz",
                 "Saccharomyces_cerevisiae.faa": "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_protein.faa.gz",
                 "Saccharomyces_cerevisiae.fna": "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.fna.gz",
                 "Saccharomyces_cerevisiae.gff3": "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/146/045/GCF_000146045.2_R64/GCF_000146045.2_R64_genomic.gff.gz",
                 "Trypanosoma_brucei.faa": "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/210/295/GCF_000210295.1_ASM21029v1/GCF_000210295.1_ASM21029v1_protein.faa.gz",
                 "Trypanosoma_brucei.fna": "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/210/295/GCF_000210295.1_ASM21029v1/GCF_000210295.1_ASM21029v1_genomic.fna.gz",
                 "Trypanosoma_brucei.gff3": "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/210/295/GCF_000210295.1_ASM21029v1/GCF_000210295.1_ASM21029v1_genomic.gff.gz",
                 "Dictyostelium_discoideum.faa": "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/004/695/GCF_000004695.1_dicty_2.7/GCF_000004695.1_dicty_2.7_protein.faa.gz",
                 "Dictyostelium_discoideum.fna": "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/004/695/GCF_000004695.1_dicty_2.7/GCF_000004695.1_dicty_2.7_genomic.fna.gz",
                 "Dictyostelium_discoideum.gff3": "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/004/695/GCF_000004695.1_dicty_2.7/GCF_000004695.1_dicty_2.7_genomic.gff.gz",
                 "Allomyces_macrogynus.faa": "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/151/295/GCA_000151295.1_A_macrogynus_V3/GCA_000151295.1_A_macrogynus_V3_protein.faa.gz",
                 "Allomyces_macrogynus.fna": "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/151/295/GCA_000151295.1_A_macrogynus_V3/GCA_000151295.1_A_macrogynus_V3_genomic.fna.gz",
                 "Allomyces_macrogynus.gff3": "ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/151/295/GCA_000151295.1_A_macrogynus_V3/GCA_000151295.1_A_macrogynus_V3_genomic.gff.gz"
          }

# Make a new temporary directory to store data files.
temp_db_dir_name = 'temporary_db_dir'
if not os.path.isdir(temp_db_dir_name):
    os.mkdir(temp_db_dir_name)

# Download all the data files via NCBI's FTP server.
for filename in datafile_dict.keys():
    url = datafile_dict[filename]
    filepath = os.path.join(temp_db_dir_name, filename)
    if not os.path.isfile(filepath):
        subprocess.call(['curl', url, '--output', filepath + '.gz'])
        subprocess.call(['gunzip', filepath + '.gz'])

CPU times: user 31.7 ms, sys: 76.5 ms, total: 108 ms
Wall time: 4min 5s


## Initiate a data directory structure
To generate a directory structure and spreadsheets for storing formatted sequence files
and metadata for each sequence file, use the 'mkdatadir' command (this takes a
single argument which is the full path that you want your new directory to be
written to):

In [12]:
%env DATADIR=AMOEBAE_Data

env: DATADIR=AMOEBAE_Data


In [13]:
%%bash
amoebae mkdatadir $DATADIR

This will prompt you to set the 'root\_amoebae\_data\_dir' variable in the
settings.py file to this new directory path so that AMOEBAE scripts can locate
your files.

This can be done as follows:

In [14]:
# Check that the path indicated in the settings file is correct.
print(settings.root_amoebae_data_dir)
assert settings.root_amoebae_data_dir == "AMOEBAE_Data"

AMOEBAE_Data


## Prepare databases for searching
To generate a directory structure and spreadsheets for storing formatted sequence files
and metadata for each sequence file, use the 'mkdatadir' command (this takes a
single argument which is the full path that you want your new directory to be
written to).

This will take at least 11 minutes, because the FASTA files need to be re-written with re-formatted sequence headers and the GFF3 files need to be converted to SQL databases using gffutils.

In [15]:
%%bash
SECONDS=0

for X in temporary_db_dir/*; do amoebae add_to_dbs $X; done

ELAPSED="Preparing sequence databases for searching took the following amount of time: $(($SECONDS / 3600))hrs $((($SECONDS / 60) % 60))min $(($SECONDS % 60))sec"
echo $ELAPSED



Building a new DB, current time: 07/13/2020 17:04:26
New DB name:   /home/vagrant/amoebae/notebooks/amoebae_tutorial_2_output/AMOEBAE_Data/Genomes/Allomyces_macrogynus.fna
New DB title:  AMOEBAE_Data/Genomes/Allomyces_macrogynus.fna
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 101 sequences in 1.37877 seconds.
Creating SSI index for AMOEBAE_Data/Genomes/Allomyces_macrogynus.fna...    done.
Indexed 101 sequences (101 names).
SSI index written to file AMOEBAE_Data/Genomes/Allomyces_macrogynus.fna.ssi


Building a new DB, current time: 07/13/2020 17:05:56
New DB name:   /home/vagrant/amoebae/notebooks/amoebae_tutorial_2_output/AMOEBAE_Data/Genomes/Arabidopsis_thaliana.faa
New DB title:  AMOEBAE_Data/Genomes/Arabidopsis_thaliana.faa
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 48265 sequences in 4.1183 seconds.
Creating SSI index for AMOEBAE_Data/Genomes/Arabidopsis_thal

In [16]:
%%bash
# List the databases now accessible by AMOEBAE.
amoebae list_dbs

Allomyces_macrogynus.fna
Arabidopsis_thaliana.faa
Arabidopsis_thaliana.fna
Dictyostelium_discoideum.faa
Dictyostelium_discoideum.fna
Saccharomyces_cerevisiae.faa
Saccharomyces_cerevisiae.fna
Trypanosoma_brucei.faa
Trypanosoma_brucei.fna


After adding each file to the AMOEBAE_Data directory, a line is added to the 0_genome_info.csv file with information describing this file. Information from this CSV file is used in downstream analysis steps, so it is important to ensure that it is accurate. 

You can view the contents of this CSV file here (empty fields are displayed as "NaN"):

In [17]:
# Find the path to the CSV file.
csv_file_path = os.path.join(os.path.join(os.environ['DATADIR'], 'Genomes'), '0_genome_info.csv')
# Load data from the CSV file using the pandas library.
df = pd.read_csv(csv_file_path)
# Display the data in an HTML table.
print("Contents of the file %s:" % csv_file_path)
display(HTML(df.to_html()))

Contents of the file AMOEBAE_Data/Genomes/0_genome_info.csv:


,Filename,Superbranch,Supergroup,Group,Species (if applicable),Taxon,Strain,Data type,Assembly,Source,Date,Citation,Note,?,Annotations file
0,Allomyces_macrogynus.fna,NaN,NaN,NaN,Allomyces macrogynus,NaN,NaN,nucl,NaN,NaN,2020/07/13,NaN,NaN,NaN,NaN
1,Allomyces_macrogynus.sql,NaN,NaN,NaN,Allomyces macrogynus,NaN,NaN,annotations,NaN,NaN,2020/07/13,NaN,NaN,NaN,NaN
2,Arabidopsis_thaliana.faa,NaN,NaN,NaN,Arabidopsis thaliana,NaN,NaN,prot,NaN,NaN,2020/07/13,NaN,NaN,NaN,NaN
3,Arabidopsis_thaliana.fna,NaN,NaN,NaN,Arabidopsis thaliana,NaN,NaN,nucl,NaN,NaN,2020/07/13,NaN,NaN,NaN,NaN
4,Arabidopsis_thaliana.sql,NaN,NaN,NaN,Arabidopsis thaliana,NaN,NaN,annotations,NaN,NaN,2020/07/13,NaN,NaN,NaN,NaN
5,Dictyostelium_discoideum.faa,NaN,NaN,NaN,Dictyostelium discoideum,NaN,NaN,prot,NaN,NaN,2020/07/13,NaN,NaN,NaN,NaN
6,Dictyostelium_discoideum.fna,NaN,NaN,NaN,Dictyostelium discoideum,NaN,NaN,nucl,NaN,NaN,2020/07/13,NaN,NaN,NaN,NaN
7,Dictyostelium_discoideum.sql,NaN,NaN,NaN,Dictyostelium discoideum,NaN,NaN,annotations,NaN,NaN,2020/07/13,NaN,NaN,NaN,NaN
8,Saccharomyces_cerevisiae.faa,NaN,NaN,NaN,Saccharomyces cerevisiae,NaN,NaN,prot,NaN,NaN,2020/07/13,NaN,NaN,NaN,NaN
9,Saccharomyces_cerevisiae.fna,NaN,NaN,NaN,Saccharomyces cerevisiae,NaN,NaN,nucl,NaN,NaN,2020/07/13,NaN,NaN,NaN,NaN


I recommended that you add any relevant information to complete this table, although this is not necessary to complete this tutorial. This can serve as a useful record for your own reference, as well as as a supplementary file in publications.

By default, AMOEBAE assumes that GFF3 annotation files will be given the same filename (besides the extension) as the nucleotide FASTA file containing the sequences that the annotations are for. For example, to retrieve annotations for assembled chromosome sequences in a file named "Arabidopsis_thaliana.fna", AMOEBAE will look for a file named "Arabidopsis_thaliana.sql" (which is generated using a file with the name "Arabidopsis_thaliana.gff3). If you wish to use different annotation files, open the CSV file in a spreadsheet program such as Excel or Open Office then copy the name of the .sql file to the row for the corresponding genomic assembly (.fna) file in the column with the header "Annotations file", and do the same for the row describing the corresponding peptide sequence (.faa) file. This allows the correct GFF3 file to be used for the assembly (.fna file) and predicted amino acid sequences (.faa).

Taxonomic information for each genome is arbitrarily divided into four hierarchical categories: "Superbranch", "Supergroup", "Group", and "Species (if applicable)". For files containing sequences or annotations for *Arabidopsis thaliana*, I would enter the values "Diaphoretickes", "Archaeplastida", "Embryophyta", and "Arabidopsis thaliana", respectively. These are arbitrary selected taxonomic groups to which Arabidopsis belongs (Adl et al., 2018). This can be useful when you have many genomes represented in the table.

# Set up queries

## Download single-sequence queries

In [18]:
%%time

# Define a dictionary with NCBI sequence accessions as keys and filenames to write
# the corresponding sequences to as values.
query_dict = {"NP_194077.1": "AP1beta_Athaliana_NP_194077.1_query.faa",
              "NP_851058.1": "AP2alpha_Athaliana_NP_851058.1_query.faa",
              "NP_974895.1": "AP2mu_Athaliana_NP_974895.1_query.faa",
              "NP_175219.1": "AP2sigma_Athaliana_NP_175219.1_query.faa",
              "NP_566961.1": "Sec12_Athaliana_NP_566961.1_query.faa",
              "NP_200929.1": "SNAP33_Athaliana_NP_200929.1_query.faa",
              "NP_193449.1": "Rab2_Athaliana_NP_193449.1_query.faa"
          }

# Make a new temporary directory to store sequence files.
temp_query_dir_name = 'temporary_query_dir'
if not os.path.isdir(temp_query_dir_name):
    os.mkdir(temp_query_dir_name)

# Loop over keys in the query_dict dictionary.
for accession in query_dict.keys():
    # Retrieve the corresponding filename from the dictionary.
    filename = query_dict[accession]
    filepath = os.path.join(temp_query_dir_name, filename)
    # Only download sequences that have not already been downloaded.
    if not os.path.isfile(filepath):
        # Download the sequence from the NCBI Protein database.
        accessions = [accession]
        url = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein&rettype=fasta&retmode=text&id=' + ','.join(accessions)
        r = requests.get(url)
        with open(filepath, 'w') as o:
            o.write(r.text)
    # Check that the sequence was actually downloaded.
    assert os.path.isfile(filepath), """The sequence with the following accession could not be downloaded from NCBI: %s\n
    Try re-running this cell.""" % accession

CPU times: user 120 ms, sys: 46.7 ms, total: 167 ms
Wall time: 4.99 s


## Prepare single-sequence queries for searching

Queries must be formatted and stored in a similar manner to genomic data files. The query files will include FASTA files containing one sequence and FASTA files containing multiple sequences.
Now we are going to generate the query files and add them to your AMOEBAE_Data/ Queries directory, in a similar way to how we added genomic data files to the AMOEBAE_Data/Genomes directory. Since you already downloaded all the peptide sequences for Arabidopsis thaliana, you can retrieve these from your downloaded data using one of the scripts in the amoebae/misc_scripts folder. First, let’s generate a query for the A. thaliana AP-1/2 beta subunit(s), which is a component of both the AP-1 and AP-2 complexes, using a representative sequence:

In [19]:
%%bash
SECONDS=0

for QUERYFILE in temporary_query_dir/*.faa; do amoebae add_to_queries $QUERYFILE; done

ELAPSED="Preparing query sequences for searching took the following amount of time: $(($SECONDS / 3600))hrs $((($SECONDS / 60) % 60))min $(($SECONDS % 60))sec"
echo $ELAPSED

Preparing query sequences for searching took the following amount of time: 0hrs 0min 5sec


In [20]:
%%bash
amoebae list_queries

AP1beta_Athaliana_NP_194077.1_query.faa
AP2alpha_Athaliana_NP_851058.1_query.faa
AP2mu_Athaliana_NP_974895.1_query.faa
AP2sigma_Athaliana_NP_175219.1_query.faa
Rab2_Athaliana_NP_193449.1_query.faa
SNAP33_Athaliana_NP_200929.1_query.faa
Sec12_Athaliana_NP_566961.1_query.faa


## Construct alignments for profile similarity searching

In [21]:
%%time

# Define a dictionary of NCBI sequence accessions and filenames to which to write the corresponding sequences.
query_title_dict = {"AP1beta": "NP_194077.1,CBI34366.3,XP_015631818.1,XP_024516549.1,OAE33273.1",
                    "AP2alpha": "NP_851058.1,XP_002270388.1,XP_015631820.1,PTQ35247.1,XP_024525508.1",
                    "AP2mu": "NP_974895.1,XP_002281297.1,XP_015627628.1,OAE25965.1,XP_002973295.1",
                    "AP2sigma": "NP_175219.1,XP_015618362.1,PTQ50284.1,XP_002275803.1,XP_024518676.1",
                    "Sec12": "NP_566961.1,XP_002262948.1,XP_015647566.1,OAE21792.1,XP_024530559.1",
                    "SNAP33": "NP_200929.1,XP_002284486.1,AAW82752.1,EFJ31467.1,OAE29824.1,XP_006270633.1,XP_006010378.1,XP_006625751.1,NP_001080510.1,XP_020370357.1,XP_015181699.1,XP_031769811.1",
                    "Rab2": "NP_193449.1,XP_003635585.2,XP_015626284.1,XP_002965710.1,PTQ28228.1"
                   }
                    

# Make a new temporary directory to store sequence files.
temp_alignment_dir_name = 'temporary_alignment_dir'
assert not os.path.isdir(temp_alignment_dir_name), """Directory already exists."""
os.mkdir(temp_alignment_dir_name)

# Download query sequences and write to multiple-sequence FASTA files.
for query_title in query_title_dict.keys():
    accession_list_string = query_title_dict[query_title]
    filepath = os.path.join(temp_alignment_dir_name, query_title + '_hmm1.faa')
    if not os.path.isfile(filepath):
        # Download the sequence from the NCBI Protein database.
        accessions = [accession_list_string]
        url = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=protein&rettype=fasta&retmode=text&id=' + ','.join(accessions)
        r = requests.get(url)
        with open(filepath, 'w') as o:
            o.write(r.text)
        # Check that the sequences were actually downloaded.
        assert os.path.isfile(filepath)


CPU times: user 129 ms, sys: 42.1 ms, total: 171 ms
Wall time: 6.62 s


In [22]:
%%bash
SECONDS=0

for X in temporary_alignment_dir/*.faa; do amoebae align_fa $X --output_format fasta; done

ELAPSED="Aligning FASTA files took the following amount of time: $(($SECONDS / 3600))hrs $((($SECONDS / 60) % 60))min $(($SECONDS % 60))sec"
echo $ELAPSED

Aligning FASTA files took the following amount of time: 0hrs 0min 6sec



MUSCLE v3.8.31 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

AP1beta_hmm1 5 seqs, max length 920, avg  length 901
00:00:00     10 MB(1%)  Iter   1  100.00%  K-mer dist pass 1
00:00:00     10 MB(1%)  Iter   1  100.00%  K-mer dist pass 2
00:00:00     16 MB(2%)  Iter   1  100.00%  Align node
00:00:00     16 MB(2%)  Iter   1  100.00%  Root alignment
00:00:00     16 MB(2%)  Iter   2  100.00%  Refine tree
00:00:00     16 MB(2%)  Iter   2  100.00%  Root alignment
00:00:00     16 MB(2%)  Iter   2  100.00%  Root alignment
00:00:00     16 MB(2%)  Iter   3  100.00%  Refine biparts
00:00:00     16 MB(2%)  Iter   4  100.00%  Refine biparts
00:00:00     16 MB(2%)  Iter   5  100.00%  Refine biparts
00:00:00     16 MB(2%)  Iter   6  100.00%  Refine biparts
00:00:00     16 MB(2%)  Iter   7  100.00%  Refine biparts
00:00:00     16 MB(2%)  Iter   8  100.00%  Refine biparts
00:00:00     16 MB(2%)

Similar to the AMOEBAE_Data/Genomes/0_genome_info.csv file, the AMOEBAE_Data/Queries/0_query_info.csv file contains information about each query file, which can be manually edited. One of the most important pieces of information is the "Query title". Different query files, such as a single FASTA sequence and an HMM, can have the same Query title if they are to be used to search for homologues or orthologues of the same protein(s).

In [23]:
# Find the path to the CSV file.
csv_file_path = os.path.join(os.path.join(os.environ['DATADIR'], 'Queries'), '0_query_info.csv')
# Load data from the CSV file using the pandas library.
df = pd.read_csv(csv_file_path)
# Display the data in an HTML table.
print("Contents of the file %s:" % csv_file_path)
display(HTML(df.to_html()))

Contents of the file AMOEBAE_Data/Queries/0_query_info.csv:


,Filename,Query title,Query source description,Query taxon (species if applicable),Query database filename (if applicable),File type,Data type,Date added,Citation
0,AP1beta_Athaliana_NP_194077.1_query.faa,AP1beta,Athaliana,-,-,faa,prot,2020/07/13,?
1,AP2alpha_Athaliana_NP_851058.1_query.faa,AP2alpha,Athaliana,-,-,faa,prot,2020/07/13,?
2,AP2mu_Athaliana_NP_974895.1_query.faa,AP2mu,Athaliana,-,-,faa,prot,2020/07/13,?
3,AP2sigma_Athaliana_NP_175219.1_query.faa,AP2sigma,Athaliana,-,-,faa,prot,2020/07/13,?
4,Rab2_Athaliana_NP_193449.1_query.faa,Rab2,Athaliana,-,-,faa,prot,2020/07/13,?
5,SNAP33_Athaliana_NP_200929.1_query.faa,SNAP33,Athaliana,-,-,faa,prot,2020/07/13,?
6,Sec12_Athaliana_NP_566961.1_query.faa,Sec12,Athaliana,-,-,faa,prot,2020/07/13,?


## Visually inspect alignments
Alignments used as queries should be visually inspected to make sure that there are no obvious errors in the alignment.

In [24]:
%%bash
for QUERYFILE in temporary_alignment_dir/*.afaa; do amoebae afa_to_nex $QUERYFILE; done
echo "Alignments to observe:"
ls temporary_alignment_dir/*.nex

Alignments to observe:
temporary_alignment_dir/AP1beta_hmm1.nex
temporary_alignment_dir/AP2alpha_hmm1.nex
temporary_alignment_dir/AP2mu_hmm1.nex
temporary_alignment_dir/AP2sigma_hmm1.nex
temporary_alignment_dir/Rab2_hmm1.nex
temporary_alignment_dir/SNAP33_hmm1.nex
temporary_alignment_dir/Sec12_hmm1.nex


## Prepare query alignments for searching

In [25]:
%%bash
SECONDS=0

for QUERYFILE in temporary_alignment_dir/*.afaa; do amoebae add_to_queries $QUERYFILE; done

ELAPSED="Preparing HMM queries from alignments took the following amount of time: $(($SECONDS / 3600))hrs $((($SECONDS / 60) % 60))min $(($SECONDS % 60))sec"
echo $ELAPSED

# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.3 (Nov 2019); http://hmmer.org/
# Copyright (C) 2019 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# input alignment file:             AMOEBAE_Data/Queries/AP1beta_hmm1_temp1.afa
# output HMM file:                  AMOEBAE_Data/Queries/AP1beta_hmm1.hmm
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# idx name                  nseq  alen  mlen eff_nseq re/pos description
#---- -------------------- ----- ----- ----- -------- ------ -----------
1     AP1beta_hmm1_temp1       5   935   899     0.46  0.595 

# CPU time: 0.83u 0.01s 00:00:00.84 Elapsed: 00:00:00.86
# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.3 (Nov 2019); http://hmmer.org/
# Copyright (C) 2019 Howard Hughes Medical Institute.
# Freely distributed under the BSD op

List queries

In [26]:
%%bash
amoebae list_queries

AP1beta_Athaliana_NP_194077.1_query.faa
AP1beta_hmm1.afaa
AP2alpha_Athaliana_NP_851058.1_query.faa
AP2alpha_hmm1.afaa
AP2mu_Athaliana_NP_974895.1_query.faa
AP2mu_hmm1.afaa
AP2sigma_Athaliana_NP_175219.1_query.faa
AP2sigma_hmm1.afaa
Rab2_Athaliana_NP_193449.1_query.faa
Rab2_hmm1.afaa
SNAP33_Athaliana_NP_200929.1_query.faa
SNAP33_hmm1.afaa
Sec12_Athaliana_NP_566961.1_query.faa
Sec12_hmm1.afaa


## Generate lists of potential redundant sequences among *A. thaliana* peptide sequences

In this tutorial, a reciprocal-best-hit search strategy will be used. If you are using a reciprocal- best-hit search strategy, then your initial round of searches will be performed using your original queries (assembled above) to search your genomes of interest. This initial round of searches will be referred to herein as "forward searches", and subsequent searches using forward search hits as queries into reference genomes will be referred to as "reverse searches".

A slight complication to this search strategy is that the NCBI RefSeq peptide sequences for the *A. thaliana* genome include alternative transcripts and lineage-specific inparalogues (as do other databases), implying that if these were retrieved as the top hits in the reverse searches instead of the original query sequence, then this would still potentially be a positive result. So, to properly interpret reverse search results it will be necessary to determine which sequences in our A. thaliana.faa file are redundant for our purposes. To do this we will use the get_redun_hits command:

In [27]:
%%bash
# Optional. Get the help output for the get_redun_hits command.
amoebae get_redun_hits -h

usage: amoebae [-h] [--csv_file CSV_FILE] [--query_name QUERY_NAME]
               [--query_list_file QUERY_LIST_FILE] [--db_name DB_NAME]
               [--db_list_file DB_LIST_FILE] [--query_title QUERY_TITLE]
               [--outdir OUTDIR]
               [--blast_report_evalue_cutoff BLAST_REPORT_EVALUE_CUTOFF]
               [--blast_max_target_seqs BLAST_MAX_TARGET_SEQS]
               [--hmmer_report_evalue_cutoff HMMER_REPORT_EVALUE_CUTOFF]
               [--hmmer_report_score_cutoff HMMER_REPORT_SCORE_CUTOFF]
               [--max_number_of_hits_to_summarize MAX_NUMBER_OF_HITS_TO_SUMMARIZE]
               [--num_threads_similarity_searching NUM_THREADS_SIMILARITY_SEARCHING]
               [--predict_redun_hit_selection]
               srch_dir

Run searches with queries to find redundant hits in databases (for
interpreting results).

positional arguments:
  srch_dir              Path to directory that will contain output directory
                        as a subdirectory.

o

In [28]:
%env REDUNHITDIR=Redundant_hits

env: REDUNHITDIR=Redundant_hits


This step will take approximately 5 minutes:

In [34]:
%%bash
SECONDS=0

# Make a directory to store information about redundant hits.
mkdir $REDUNHITDIR

# Write a file listing names of query files to be used.
amoebae list_queries > $REDUNHITDIR/queries.txt

# Use AMOEBAE to retrieve potential redundant hit sequences.
amoebae get_redun_hits $REDUNHITDIR/redun_hits_1 \
            --query_list_file $REDUNHITDIR/queries.txt \
            --db_name Arabidopsis_thaliana.faa \
            --csv_file $REDUNHITDIR/redun_hits_1/0_redun_hits_1.csv

ELAPSED="Retrieving potentially redundant sequences took the following amount of time: $(($SECONDS / 3600))hrs $((($SECONDS / 60) % 60))min $(($SECONDS % 60))sec"
echo $ELAPSED

DONE!


Edit spreadsheet to classify hits as redundant or not before
        proceeding (modify values in the 'Positive/redundant (+) or negative
        (-) hit for queries with query title' column):

	Redundant_hits/redun_hits_1/0_redun_hits_1.csv

Retrieving potentially redundant sequences took the following amount of time: 0hrs 1min 7sec


This will output a directory in the Redundant_hits folder with a .csv file. This file contains a summary of BLASTP or HMMer search results for searches with the specified queries into the *A. thaliana* predicted proteins. 

It should be apparent upon inspection of the ranking of hits and comparison of the associated E-values which hits are likely redundant with your queries (see cell below). To tell AMOEBAE which hits you want to consider as redundant for the purposes of downstream steps, the values in the column with the header "Positive/redundant (+) or negative (-) hit for queries with query title (edit this column)" must be changed from ’-’ to ’+’ for hits that are redundant (this will be done automatically for this tutorial (see below).

In [35]:
# Find the path to the CSV file.
csv_file_path = glob.glob(os.path.join('Redundant_hits', os.path.join('redun_hits_1', '0_redun_hits_1.csv')))[0]
# Load data from the CSV file using the pandas library.
df = pd.read_csv(csv_file_path)
# Display the data in an HTML table.
print("Contents of the file %s:" % csv_file_path)
display(HTML(df.to_html()))

Contents of the file Redundant_hits/redun_hits_1/0_redun_hits_1.csv:


,Query Title,Query File,Database File,Search Method,Positive/redundant (+) or negative (-) hit for queries with query title (edit this column),Hit Number,E-value,Order of magnitude of E-value difference compared to top hit,Length as percent of top hit sequence length,Accession,Description,Unnamed: 11
0,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,-,1,0.000000e+00,0,100,NP_194077.1,NP_194077.1 Adaptin family protein [Arabidopsis thaliana],NaN
1,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,-,2,0.000000e+00,0,100,NP_192877.1,NP_192877.1 Adaptin family protein [Arabidopsis thaliana],NaN
2,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,-,3,0.000000e+00,0,89,NP_001328014.1,NP_001328014.1 Adaptin family protein [Arabidopsis thaliana],NaN
3,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,-,4,0.000000e+00,0,103,NP_001190701.1,NP_001190701.1 Adaptin family protein [Arabidopsis thaliana],NaN
4,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,-,5,2.418810e-110,190,94,NP_196710.1,NP_196710.1 adaptin family protein [Arabidopsis thaliana],NaN
5,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,-,6,3.470670e-110,191,95,NP_001190290.1,NP_001190290.1 adaptin family protein [Arabidopsis thaliana],NaN
6,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,-,7,1.576930e-105,195,89,NP_001330519.1,NP_001330519.1 adaptin family protein [Arabidopsis thaliana],NaN
7,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,-,8,6.586930e-30,271,125,NP_974443.4,NP_974443.4 protein affected trafficking 2 [Arabidopsis thaliana],NaN
8,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,-,9,2.023720e-22,278,111,NP_567022.1,NP_567022.1 protein affected trafficking 2 [Arabidopsis thaliana],NaN
9,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,-,10,1.004840e-10,290,109,NP_001328341.1,"NP_001328341.1 Coatomer, beta subunit [Arabidopsis thaliana]",NaN


## Identify redundant sequences

In [38]:
%time
# Define a dictionary with query titles as keys and lists of sequence IDs as values, where the IDs are for A. thaliana sequences that are redundant with the original A. thaliana query sequence.
redun_seq_dict = {"AP1beta":  ["NP_194077.1",
                               "NP_192877.1",
                               "NP_001328014.1",
                               "NP_001190701.1"
                               ],
                  
                  "AP2alpha": ["NP_851058.1",
                               "NP_851057.1",
                               "NP_197669.1",
                               "NP_001330971.1",
                               "NP_001330970.1",
                               "NP_001330969.1",
                               "NP_197670.1",
                               "NP_001330127.1"
                               ],
                  
                  "AP2mu":    ["NP_974895.1",
                               "NP_199475.1"
                               ],
                  
                  "AP2sigma": ["NP_175219.1"
                               ],
                  
                  "Sec12":    ["NP_566961.1",
                               "NP_568738.1",
                               "NP_680414.1",
                               "NP_178256.1"
                               ],
                  
                  "SNAP33":   ["NP_200929.1",
                               "NP_001332102.1",
                               "NP_172842.1",
                               "NP_001318998.1",
                               "NP_196405.1",
                               "NP_001318503.1"
                               ],
                  
                  "Rab2":     ["NP_193449.1",
                               "NP_193450.1",
                               "NP_195311.1",
                               "NP_001078499.1"
                               ]
                   }


# Identify path to redundant seqs CSV file.
redundant_seqs_csv = glob.glob(os.path.join('Redundant_hits', os.path.join('redun_hits_1', '0_redun_hits_1.csv')))[0]

# Define path for new modified redundant seqs CSV file.
redundant_seqs_csv2 = redundant_seqs_csv.rsplit("_", 1)[0] + '_2.csv'

# Open the redundant seqs CSV file, and a new one.
with open(redundant_seqs_csv) as infh, open(redundant_seqs_csv2, 'w') as o:
    # Loop over lines in the CSV file.
    for i in infh:
        if not i.startswith("Query Title"):
            # Identify query title in line.
            line_query_title = i.split(',')[0].strip()
            # Identify accession/id for sequence hit represented in this row.
            line_accession = i.split(',')[9].strip().strip('\"')
            # Loop over keys (query titles) in the redundant seqs dictionary.
            query_title_in_keys = False
            for query_title in redun_seq_dict.keys():
                if line_query_title == query_title:
                    query_title_in_keys = True
                    #print('YYY')
                    #print(line_accession)
                    #print(redun_seq_dict[line_query_title])
                    # Determine whether the accession is a redundant accession.
                    for acc in redun_seq_dict[line_query_title]:
                        #print(line_accession, acc)
                        if line_accession == acc:
                            # Change the - to + so that the accession will be included in the list of redundant accessions used by AMOEBAE.
                            i = ','.join(i.split(',')[:4]) + ',+,' + ','.join(i.split(',')[5:])
                            break
                # Break loop if the corresponding query title was found already.
                if query_title_in_keys:
                    break
            # Check that a query title could be recognized as one that is a key in the dictionary.
            assert query_title_in_keys, """Could not find query title %s in dictionary.""" % line_query_title
        # Write (modified) line to new CSV file.
        o.write(i)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.77 µs


Observe how the file has been modified.

In [39]:
# Find the path to the CSV file.
csv_file_path = glob.glob(os.path.join('Redundant_hits', os.path.join('redun_hits_1', '0_redun_hits_2.csv')))[0]
# Load data from the CSV file using the pandas library.
df = pd.read_csv(csv_file_path)
# Display the data in an HTML table.
print("Contents of the file %s:" % csv_file_path)
display(HTML(df.to_html()))

Contents of the file Redundant_hits/redun_hits_1/0_redun_hits_2.csv:


,Query Title,Query File,Database File,Search Method,Positive/redundant (+) or negative (-) hit for queries with query title (edit this column),Hit Number,E-value,Order of magnitude of E-value difference compared to top hit,Length as percent of top hit sequence length,Accession,Description,Unnamed: 11
0,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,+,1,0.000000e+00,0,100,NP_194077.1,NP_194077.1 Adaptin family protein [Arabidopsis thaliana],NaN
1,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,+,2,0.000000e+00,0,100,NP_192877.1,NP_192877.1 Adaptin family protein [Arabidopsis thaliana],NaN
2,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,+,3,0.000000e+00,0,89,NP_001328014.1,NP_001328014.1 Adaptin family protein [Arabidopsis thaliana],NaN
3,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,+,4,0.000000e+00,0,103,NP_001190701.1,NP_001190701.1 Adaptin family protein [Arabidopsis thaliana],NaN
4,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,-,5,2.418810e-110,190,94,NP_196710.1,NP_196710.1 adaptin family protein [Arabidopsis thaliana],NaN
5,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,-,6,3.470670e-110,191,95,NP_001190290.1,NP_001190290.1 adaptin family protein [Arabidopsis thaliana],NaN
6,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,-,7,1.576930e-105,195,89,NP_001330519.1,NP_001330519.1 adaptin family protein [Arabidopsis thaliana],NaN
7,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,-,8,6.586930e-30,271,125,NP_974443.4,NP_974443.4 protein affected trafficking 2 [Arabidopsis thaliana],NaN
8,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,-,9,2.023720e-22,278,111,NP_567022.1,NP_567022.1 protein affected trafficking 2 [Arabidopsis thaliana],NaN
9,AP1beta,AP1beta_Athaliana_NP_194077.1_query.faa,Arabidopsis_thaliana.faa,blastp 2.9.0+,-,10,1.004840e-10,290,109,NP_001328341.1,"NP_001328341.1 Coatomer, beta subunit [Arabidopsis thaliana]",NaN


# Run forward searches

To begin searching, make a new folder to contain search results, and write text files listing the names (not full paths) of FASTA files you want to use as queries and those that you want to search in.

In [ ]:
%env SRCHRESDIR=AMOEBAE_Search_Results_1

In [ ]:
%%bash
# Make a new directory to contain search results.
mkdir $SRCHRESDIR
# Write query and database list files.
amoebae list_queries > $SRCHRESDIR/queries.txt
amoebae list_dbs > $SRCHRESDIR/databases.txt

Set up searches using the setup_fwd_srch command:

In [ ]:
%%bash
# Optional. Get the help output for the setup_fwd_srch command.
amoebae setup_fwd_srch -h

In [ ]:
%env FWDSRCHDIR=fwd_srch_1

In [ ]:
%%bash
# Set up forward searches.
amoebae setup_fwd_srch $SRCHRESDIR\
                       $SRCHRESDIR/queries.txt\
                       $SRCHRESDIR/databases.txt\
                       --outdir $SRCHRESDIR/$FWDSRCHDIR

This will output a new sub-directory with a name that starts with "fwd_srch_". Now run the searches with this directory as input via the run_fwd_srch command. Forward search criteria may be selected at this point (view the relevant optional arguments via the -h option).

In [ ]:
%%bash
tree $SRCHRESDIR

In [ ]:
%%bash
SECONDS=0

# Run forward searches. This could take a while.
amoebae run_fwd_srch $SRCHRESDIR/$FWDSRCHDIR

ELAPSED="Running forward searches took the following amount of time: $(($SECONDS / 3600))hrs $((($SECONDS / 60) % 60))min $(($SECONDS % 60))sec"
echo $ELAPSED

This will run BLASTP or HMMer for searches into the .faa files (depending on whether queries are single- or multi-fasta), or TBLASTN for searches into the .fna files with any single-fasta queries.

# Summarize forward search results

Now we can generate a summary of the raw output files. Important criteria may be customized here as well. Specifically the forward search E-value threshold, and the maximum number of nucleotide bases allowed between TBLASTN HSPs to be considered part of the same gene (view optional arguments via the -h option).

In [ ]:
%%bash
amoebae sum_fwd_srch -h

In [ ]:
%%time
# Summarize forward search results in a CSV file.
# ***Note that only the top 5 hits for each individual search will be reported, as specified here. 
# This is simply to save time, and previous analyses have confirmed that the number of positive hits will not exceed 5 for any of the searches.
!amoebae sum_fwd_srch $SRCHRESDIR/$FWDSRCHDIR\
                     $SRCHRESDIR/$FWDSRCHDIR'_sum.csv'\
                     --max_hits_to_sum 5 \
                     --max_gap_between_tblastn_hsps 1000
                    

Examine the resulting CSV file. Note that maximum E-value cutoffs, and other criteria were applied as specified.

In [ ]:
# Load data from the CSV file using the pandas library.
df = pd.read_csv(os.path.join(os.environ['SRCHRESDIR'],os.environ['FWDSRCHDIR']) + '_sum.csv_out.csv')
# Display the data in an HTML table.
display(HTML(df.to_html()))

Subsequent steps in this tutorial will result in copies of this spreadsheet with appended columns to describe further results relevant to each forward search hit.

# Run reverse searches

Now, to determine which of the "forward hits" in these search results are really specific to our original A. thaliana queries, let’s search with these hits as queries back into the A. thaliana genome (i.e., perform "reverse" searches).

Similar to the forward searches, we need to first set up the reverse search directory:



In [ ]:
%env REVSRCHDIR=rev_srch_1

In [ ]:
%%bash
amoebae setup_rev_srch -h

In [ ]:
%%bash
# Important: the --aasubseq option is used here.
amoebae setup_rev_srch $SRCHRESDIR\
                       $SRCHRESDIR/$FWDSRCHDIR'_sum.csv_out.csv'\
                       Arabidopsis_thaliana.faa\
                       --outdir $SRCHRESDIR/$REVSRCHDIR\
                       --aasubseq

This will output a new directory with "rev_srch_" and a timestamp in the name. Run reverse searches using the path to this directory as an input:

In [ ]:
%%bash
# View reverse search directory contents.
#tree $SRCHRESDIR/$REVSRCHDIR

Running reverse searches will take approximately 5 minutes.

In [ ]:
%%time
!amoebae run_rev_srch $SRCHRESDIR/$REVSRCHDIR

# Summarize reverse search results

Now append columns summarizing the results of these reverse searches to our CSV file. This is where the file listing redundant hits for each query title is used. Also, a criterion is applied here based on the order of magnitude difference in E-value between the original query (or redundant hits) in the reverse search results compared to other hits (if present), and this can be optionally modified (view optional arguments via the -h option).

This could take a while.

In [ ]:
%%bash
amoebae sum_rev_srch -h

Summarizing reverse search results should take approximately 5 minutes.

In [ ]:
%%bash
SECONDS=0

# Important: The --aasubseq option is used here, because it was used when the setup_rev_srch command was run above.
CSVLIST=($REDUNHITDIR/redun_hits_*/0_redun_hits_*_2.csv)
amoebae sum_rev_srch $SRCHRESDIR/$FWDSRCHDIR'_sum.csv_out.csv'\
                     $SRCHRESDIR/$REVSRCHDIR\
                     --redun_hit_csv ${CSVLIST[-1]}\
                     --aasubseq\
                     --min_evaldiff 2
                     
                     
ELAPSED="Summarizing these results took the following amount of time: $(($SECONDS / 3600))hrs $((($SECONDS / 60) % 60))min $(($SECONDS % 60))sec"
echo $ELAPSED

By default, this will output a CSV file with the same path as the forward search summary CSV file, but with a "_1" added before the filename extension. Examine the resulting CSV file.
You could run additional reverse searches into different files, appending columns to the same summary spreadsheet. Reverse searches into the A. thaliana peptide sequences is all that is necessary for this tutorial.

Next run the interp_srchs command to do an additional interpretation of the results (if reverse searches into multiple reference databases were performed then this would be done following summarization of all the reverse searches). Again, customized criteria may be applied at this point using the optional arguments.


In [ ]:
%%bash
amoebae interp_srchs $SRCHRESDIR/$FWDSRCHDIR'_sum.csv_out_1.csv'

Again, examine the resulting CSV file to see whether the results match your expectations. You will notice that the results in this file do not account for the fact that the HMMer, BLASTP, and TBLASTN hits are redundant in many cases as might be expected if each of these search algorithms were effective.

# Determine which positive hits are redundant

We need to determine which hits likely correspond to the same loci based on having identical accessions or being associated with the same locus in the GFF3 annotation file, or likely represent distinct paralogous gene loci based on sequence similarity in a multiple sequence alignment (see Larson et al. (2019) for explanation of how these are identified).
To do this, first we will append a column listing what alignment to use (by default it will be the alignments that are used as queries for the corresponding query title):


In [ ]:
%%bash
amoebae find_redun_seqs -h

In [ ]:
%%bash
CSVLIST=( $SRCHRESDIR/${FWDSRCHDIR}_sum.csv_out_1_interp_*.csv )
amoebae find_redun_seqs ${CSVLIST[-1]} --add_ali_col

Now identify distinct paralogues. This should take about 30 minutes.

In [ ]:
%%bash
SECONDS=0

CSVLIST=( $SRCHRESDIR/${FWDSRCHDIR}_sum.csv_out_1_interp_*_with_ali_col.csv )

amoebae find_redun_seqs ${CSVLIST[-1]}

ELAPSED="Finding redundant sequences took the following amount of time: $(($SECONDS / 3600))hrs $((($SECONDS / 60) % 60))min $(($SECONDS % 60))sec"
echo $ELAPSED

This will output another copy of the CSV file with additional columns. Take some time to decide whether you agree with the exclusion of some of the hits, as indicated in the appended columns. You may wish to include an abridged version of this type of final output table as a supplementary file for publications.

In [ ]:
# Load data from the CSV file using the pandas library.
csv_file = glob.glob(os.path.join(os.environ['SRCHRESDIR'],'*_paralogue_count_*.csv'))[0]
df = pd.read_csv(csv_file)
# Display the data in an HTML table.
display(HTML(df.to_html()))

# Plot the final search results

Finally, we can plot the results of the searches. To customize the organization of the output coulson plot, an additional input CSV file may be optionally provided here. This file simply contains the names of protein complexes in the first column and query titles for proteins that you want to include in each complex in the second column (see example file provided with this tutorial).

In [ ]:
%%bash
# Write a file indicating how columns in the output coulson plot should be constructed.
echo \
"AP-2,AP1beta
AP-2,AP2alpha
AP-2,AP2mu
AP-2,AP2sigma
COPII,Sec12
SNAREs,SNAP33
Rabs,Rab2" > $SRCHRESDIR/complex_info_1.csv

In [ ]:
# Write a file indicating the order in which results should be displayed.
with open(os.path.join(os.environ['SRCHRESDIR'], "databases.txt")) as infh,\
open(os.path.join(os.environ['SRCHRESDIR'], "coulson_row_order.txt"), 'w') as o:
    lines = infh.readlines()
    order_list = ["Arabidopsis",
                  "Saccharomyces",
                  "Allomyces",
                  "Dictyostelium",
                  "Trypanosoma"
                 ]
    for genus in order_list:
        for line in lines:
            if line.startswith(genus):
                o.write(line)

In [ ]:
%%bash
amoebae plot -h

In [ ]:
%%bash
SECONDS=0 

CSVLIST=( $SRCHRESDIR/${FWDSRCHDIR}_sum.csv_out_1_interp_*_with_ali_col_paralogue_count_*.csv )
amoebae plot ${CSVLIST[-1]}\
             --complex_info $SRCHRESDIR/complex_info_1.csv\
             --row_order $SRCHRESDIR/coulson_row_order.txt\
             --out_pdf $SRCHRESDIR/plot.pdf

ELAPSED="Plotting these results took the following amount of time: $(($SECONDS / 3600))hrs $((($SECONDS / 60) % 60))min $(($SECONDS % 60))sec"
echo $ELAPSED

Examine the resulting PDF files. Your coulson plot should look something like that in Figure 1. Compare with the results of searches for AP-2 subunits published by Manna et al. (2013), Barlow et al. (2014), and Larson et al. (2019). You will need to customize formatting of coulson plots output by the ’plot’ command using software such as Adobe Illustrator.

In [ ]:
Image(filename=os.path.join(os.environ['SRCHRESDIR'], "plot_coulson_both.png"), width="600px") 

**Figure 1**: A coulson plot summarizing similarity search results for AP-2 complex subunits in *Trypanosoma brucei gambiense* and *Saccharomyces cerevisiae* peptide and nucleotide sequences using *Arabidopsis thaliana* queries and Hidden Markov Models generated from alignments of embryophyte orthologues. BLASTP and TBLASTN were used to search peptide and nucleotide sequences, respectively, with single sequence queries, and the HMMer3 package was used to perform profile searches. Subplot sectors with blue fill indicate that one or more sequences were found to meet the search criteria applied (the numbers of paralogues are shown in the relevant subplot sectors, were greater than one). Note that the ancestral eukaryotic AP-1 and AP-2 complexes shared a single beta subunit (Dacks et al., 2008). This is why identified "AP1beta" orthologues are shown as a component of the AP-2 complex here, even though *T. brucei* lacks an AP-2 complex (Manna et al., 2013). These results are comparable to the relevant results published by Manna et al. (2013), Barlow et al. (2014), and Larson et al. (2019).

# Interpretation and re-analysis

It should be clear that AMOEBAE identifies "positive" and "negative" results simply by applying criteria that the user specifies. So, it is entirely the users responsibility to select appropriate criteria and interpret the results critically. No similarity search criteria, however carefully chosen, will guarantee absence of both type I and type II errors regarding identification and classification of homologous sequences. This applies to any software or workflow that you could use for similarity searching.

Points to consider regarding interpretation of the results of the analysis in this tutorial include the following:

- The BLASTP and HMMer searches (both followed by reverse BLASTP searches) yielded the similar results in this analysis.


- The TBLASTN searches were able to identify most of the genes represented by the peptide sequences identified by BLASTP and HMMer searches.


- The failure to identify a Sec12 orthologue in *S. cerevisiae* is obviously a false-negative, because "Sec12" was first identified and characterized in yeast. Compare with results reported by Schlacht and Dacks (2015). Similarly, the failure to identify a SNAP33 (Qbc-SNARE) orthologue in *S. cerevisiae* is a false-negative as well. In this case, the *S. cerevisiae* Qbc-SNARE orthologues Sec9 and Spo20 are retrieved by the SNAP33 HMM, but, because they are divergent, these do not retrieve any hits when used as BLASTP queries to search *A. thaliana* proteins.


- The results for Rab2 illustrate a limitation of the type of reciprocal-best-hit search strategy employed in this tutorial. A positive hit for Rab2 was identified in *Allomyces macrogynus* using these methods. However, Elias et al. (2012) did not identify a Rab2 orthologue *A. macrogynus* in their comprehensive analysis. The positive hit in the analysis herein is a false positive result that occurs due to the absence of Rab4 (a close relative of Rab2) in *A. thaliana*. This example highlights the importance of following up similarity searching with phylogenetic analysis, which compares identified sequences to many homologues simultaneously.


- A TBLASTN hit in the *A. thaliana* chromosome 5 (NC_003076.8) met the forward and reverse search criteria, but was excluded because the translation of the region that aligned to the query was only 50 amino acids long (this sequence also contained stop codons). If you look on the NCBI genome browser for *A. thaliana* you will see that this region on chromosome 5 (as indicated in the summary CSV file) corresponds to a pseudogene for AP-2 sigma with the gene ID AT5G42568.


- The two *A. thaliana* AP-1/2 beta paralogues and the two *S. cerevisiae* paralogues are brassicalid and fungal inparalogues, respectively, which arose from independent gene duplications. Phylogenetic analysis would be required to determine this (see Larson et al. (2019) and Barlow et al. (2014)).


- An *Arabidopsis thaliana* AP-2 mu splice variant was excluded after running the ’find_redun_seqs’ command, because it was found to be encoded by the same gene as the other splice variant based on information in the GFF3 annotation file.


- An *A. thaliana* AP-2 alpha gene was excluded after running the ’find_redun_seqs’ command, because it shows over 98% identity with the other AP-2 alpha gene. The summary CSV file indicates which file contains an alignment of these two sequences (see Larson et al. (2019) for relevant discussion).


If the analysis in this tutorial were a project you were working on for publication, then upon completing the above analysis steps you work would have only just begun. Careful inspection of the summary CSV file will reveal that minor adjustments to the search criteria would cause the analysis to yield different results. Moreover, there are many different possibilities that would lead to innacurate results based on the criteria applied in the above analysis. A comprehensive discussion of this is beyond the scope of this tutorial. In general, I recommend that you take an iterative approach to analysis involving adjustment of search criteria and re-analysis to include sequences that you know are homologues of interest, but to exclude those that you know are not homologues of interest.

To generate an alignment of homologous sequences identified using AMOEBAE, use the ’csv_to_fasta’ command to generate FASTA files for alignment, and then align using your preferred software (e.g., MUSCLE or MAFFT). For visually assessing the sequences for possible issues such as contrasting domain topologies, you may wish to generate FASTA files including all your forward search results for each query title. If you are planning to run a phylognetic analysis, you may wish to generate a FASTA file with only those sequences that match all your search criteria, and with abbreviated sequence headers the csv_to_fasta comand as follows:

In [ ]:
%%bash
CSVLIST=( $SRCHRESDIR/${FWDSRCHDIR}_sum.csv_out_1_interp_*_with_ali_col_paralogue_count_*.csv )

amoebae csv_to_fasta ${CSVLIST[-1]} --abbrev --split_by_query_title

# Delete search output files (optional)

In [ ]:
%%bash
# Delete temporary files.
#rm -r temporary_alignment_dir
#rm -r temporary_db_dir
#rm -r temporary_query_dir

In [ ]:
%%bash
# Delete all data and results files (WARNING you may want to keep these!).
#rm -r $DATADIR
#rm -r $SRCHRESDIR
#rm -r $REVSRCHDIR

# Where to go from here?

First, try modifying the parameters and observe how that changes the results. Then customize this notebook to search with different queries in different genomes, or run amoebae from the command line.

# References

Barlow, L.D., Dacks, J.B., Wideman, J.G., 2014. From all to (nearly) none: Tracing adaptin evolution in Fungi. Cellular Logistics 4, e28114. https://doi.org/10.4161/cl.28114

Dacks, J.B., Poon, P.P., Field, M.C., 2008. Phylogeny of endocytic components yields insight into the process of nonendosymbiotic organelle evolution. Proceedings of the National Academy of Sciences 105, 588–593. https://doi.org/10.1073/pnas.0707318105

Elias, M., Brighouse, A., Gabernet-Castello, C., Field, M.C., Dacks, J.B., 2012. Sculpting the endomembrane system in deep time: high resolution phylogenetics of Rab GTPases. Journal of Cell Science 125, 2500–2508. https://doi.org/10.1242/jcs.101378

Hirst, J., D. Barlow, L., Francisco, G.C., Sahlender, D.A., Seaman, M.N.J., Dacks, J.B., Robinson, M.S., 2011. The Fifth Adaptor Protein Complex. PLoS Biology 9, e1001170. https://doi.org/10.1371/journal.pbio.1001170

Larson, R.T., Dacks, J.B., Barlow, L.D., 2019. Recent gene duplications dominate evolutionary dynamics of adaptor protein complex subunits in embryophytes. Traffic 20, 961–973. https://doi.org/10.1111/tra.12698

Manna, P.T., Kelly, S., Field, M.C., 2013. Adaptin evolution in kinetoplastids and emergence of the variant surface glycoprotein coat in African trypanosomatids. Molecular Phylogenetics and Evolution 67, 123–128. https://doi.org/10.1016/j.ympev.2013.01.002

Robinson, M.S., 2004. Adaptable adaptors for coated vesicles. Trends in Cell Biology 14, 167–174. https://doi.org/10.1016/j.tcb.2004.02.002

Schlacht, A., Dacks, J.B., 2015. Unexpected Ancient Paralogs and an Evolutionary Model for the COPII Coat Complex. Genome Biology and Evolution 7, 1098–1109. https://doi.org/10.1093/gbe/evv045


# Checkpoint notebook

In [ ]:
%%javascript
// Save and checkpoint the current notebook (same as doing it manually through the GUI).
require(["base/js/namespace"],function(Jupyter) {
    Jupyter.notebook.save_checkpoint();
});

# Print this notebook

In [ ]:
# Define author name for this notebook.
author_name = ""

# Define title of this notebook.
notebook_title = 'amoebae_tutorial_2'.replace('_', ' ')

In [ ]:
# Import modules.
import os
from string import Template

# Write a latex template file for converting this notebook to latex (as an intermediate to PDF).
latex_template_string = Template(r"""
((*- extends 'article.tplx' -*))

((* block author *))
\author{$an}
((* endblock author *))

((* block title *))
\title{$nt}
((* endblock title *))
""")
latex_file_contents =\
latex_template_string.substitute(an=author_name,
                                 nt=notebook_title
                                )
latex_template_file_path = 'latex_template.tplx'
with open(latex_template_file_path, 'w') as o:
    o.write(latex_file_contents)

# Convert notebook to PDF (with latex as an intermediate to process bibtex citations, etc.).
!jupyter nbconvert ../amoebae_tutorial_2.ipynb --to pdf --template {latex_template_file_path}

# Remove latex template file and bibtex file.
os.remove(latex_template_file_path)